In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
 
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
 
 
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf

In [ ]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

In [ ]:
import numpy as np
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
from keras.layers.advanced_activations import LeakyReLU
from keras.datasets import cifar10
(train_features, train_labels), (test_features, test_labels) = cifar10.load_data()
model = Sequential()

model.add(Conv2D(filters=16, kernel_size=(2, 2), padding="same", activation="relu", input_shape=(train_features.shape[1:])))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))

model.add(Conv2D(filters=32, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))

model.add(Conv2D(filters=64, kernel_size=(4, 4), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))

model.add(Flatten())

model.add(Dense(25600, activation="relu"))
model.add(Dense(25600, activation="relu"))
model.add(Dense(25600, activation="relu"))
model.add(Dense(25600, activation="relu"))
model.add(Dense(10, activation="softmax"))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(train_features, train_labels, validation_split=0.2, epochs=10, batch_size=128, verbose=1)

In [ ]:
!ps ax | grep python

In [ ]:
!nvidia-smi

In [ ]:
!kill -9 -1

In [2]:
# Load the Drive helper and mount
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys

!test -d PyTorch-Image-Dehazing-master || git clone https://github.com/goldenbili/PyTorch-Image-Dehazing-master
if not 'PyTorch-Image-Dehazing-master' in sys.path:
  sys.path += ['PyTorch-Image-Dehazing-master']
%cd PyTorch-Image-Dehazing-master/
!pwd  

In [6]:
!unzip -q /content/drive/MyDrive/AOD-Net/new_mntrain.zip -d /content/drive/MyDrive/AOD-Net

In [ ]:
!rm -r /content/PyTorch-Image-Dehazing-master/

In [ ]:
!wget http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip
!wget http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_valid_HR.zip
!unzip -q DIV2K_train_HR.zip -d data
!unzip -q DIV2K_valid_HR.zip -d data
!rm DIV2K_train_HR.zip
!rm DIV2K_valid_HR.zip

In [ ]:
!rm -r data/

In [ ]:
!mkdir ./data
!cp ../drive/MyDrive/AOD-Net/original_image.zip ./data/
!unzip -q ./data/original_image.zip -d data
!rm ./data/original_image.zip

In [4]:
from datetime import datetime
now = datetime.now()
snapshotPaht = "/content/drive/MyDrive/AOD-Net/snapshot-"+datetime.strftime(now,'%Y-%m-%d_%H:%M:%S')
!mkdir $snapshotPaht
sampleOutputPaht = "/content/drive/MyDrive/AOD-Net/sampleOutputPaht-"+datetime.strftime(now,'%Y-%m-%d_%H:%M:%S')
!mkdir $sampleOutputPaht
snapshotPaht = snapshotPaht + '/'
sampleOutputPaht = sampleOutputPaht + '/'

In [ ]:
!cd /content/PyTorch-Image-Dehazing-master/

In [ ]:
!cd PyTorch-Image-Dehazing-master/

In [ ]:
!python -c "import torch; print(torch.__version__)"

1.7.0+cu101


In [ ]:
!python -c "import torchvision; print(torchvision.__version__)"

0.8.1+cu101


In [ ]:
!pip install torchvision==0.5

In [ ]:
!pip install torch==1.4

In [ ]:
!pip install piq

In [7]:
background_color = 'mix'
line_color = 'random'
image_path = '/content/drive/MyDrive/AOD-Net/test_images/' + background_color + '/' + line_color + '/'

In [ ]:
print(image_path)
print(snapshotPaht)
print(sampleOutputPaht)

In [ ]:
! python make_image.py \
  --width 32 \
  --height 32 \
  --background $background_color \
  --line_color $line_color \
  --num_images 1000 \
  --load_path $image_path

In [8]:
! python train.py \
  --use_gpu 1 \
  --orig_images_path /content/drive/MyDrive/AOD-Net/new_mntrain/ \
  --snapshots_folder $snapshotPaht \
  --sample_output_folder $sampleOutputPaht \
  --snap_train_data dehazer.pth \
  --resize 0 \
  --num_workers 4 \
  --num_epochs 50

snapshots_folder:/content/drive/MyDrive/AOD-Net/snapshot-2021-01-26_07:15:28/
snap_train_data:dehazer.pth
sample_output_folder:/content/drive/MyDrive/AOD-Net/sampleOutputPaht-2021-01-26_07:15:28/
conf.resize:
0
Traceback (most recent call last):
  File "train.py", line 404, in <module>
    train(conf)
  File "train.py", line 95, in train
    dehaze_net.load_state_dict(torch.load(config.snapshots_folder + config.snap_train_data))
  File "/usr/local/lib/python3.6/dist-packages/torch/serialization.py", line 581, in load
    with _open_file_like(f, 'rb') as opened_file:
  File "/usr/local/lib/python3.6/dist-packages/torch/serialization.py", line 230, in _open_file_like
    return _open_file(name_or_buffer, mode)
  File "/usr/local/lib/python3.6/dist-packages/torch/serialization.py", line 211, in __init__
    super(_open_file, self).__init__(open(name, mode))
FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/AOD-Net/snapshot-2021-01-26_07:15:28/dehazer.pth'


In [ ]:
!cp -R ./epochs/ ../drive/My\ Drive/

In [ ]:
!cp -R ./training_results ../drive/My\ Drive/

In [ ]:
!cp -R ./training_results ../drive/My\ Drive/